In [91]:
#load packages
import openai
import os
from dotenv import load_dotenv
load_dotenv()

True

In [92]:
import pickle

In [93]:
import instructor
from pydantic import BaseModel, Field

In [94]:
import enum

In [95]:
from openai import OpenAI

In [96]:
from define_questions import questions

In [97]:
# a few ways to think about accuracy
# 1. simple equal
# 2. inclusion test
# 3. write a function and have chatgpt three check it

In [98]:
# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

In [99]:
def check_accurate_equal(str1,str2):
    if str1 == str2:
        return True
    else:
        return False

def check_accurate_include(str1,str2):
    if (str1 in str2)|(str2 in str1):
        return True
    else:
        return False


class chatty_correct_type(str, enum.Enum):
    """Possible Values for chatty_correct"""
    Yes = True
    No = False
    Maybe = "Maybe"


class chatty_correct(BaseModel):
    """
    Answer to the question: "Is the answer correct?"
    """
    val: chatty_correct_type
    why: str = Field(
        description="explain why you think the answer is correct or not"
    )

def check_chatty_correct(str1,str2):

    result = client.chat.completions.create(
    model="gpt-4o",
    response_model=chatty_correct,
    temperature=0.0,
    messages=[{"role": "user",
     "content": f"""***{str1}*** is the standard answer and ***{str2}*** is provided by a human. 
     Check if the human answer and the standard answer agree in principle.
     """,}],
    )
    return result

In [100]:
data_dir = '../data/bankruptcy_dockets/documents'
folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]

In [101]:
#save a big dataframe

In [116]:
import sys

sys.setrecursionlimit(20000)

In [123]:
def read_row(folder):
    #read the pickle
    row = pickle.load(open(f'../data/bankruptcy_dockets/output/persist_{folder}/results_{'-'.join(folder.split(' '))}.pkl','rb'))
    row_result = {}
    print(folder)

    for question_name,val in questions.items():
        check_response = check_chatty_correct(row[question_name],
        row[f'{question_name}_response']).dict()
        row_result[question_name] = row[question_name]
        row_result[f'{question_name}_response'] = row[f'{question_name}_response']
        row_result[f'{question_name}_correct'] = check_response['val']
        row_result[f'{question_name}_why'] = check_response['why']

    return row, row_result

In [129]:
from pqdm.processes import pqdm

In [131]:
results = pqdm(folders,read_row,3) 

QUEUEING TASKS | : 100%|██████████| 20/20 [00:00<00:00, 552.67it/s]


Armstrong FlooringEndo International PlcBasic Energy Services





PROCESSING TASKS | :   5%|▌         | 1/20 [00:38<12:16, 38.77s/it]

CBL & Associates Properties


PROCESSING TASKS | :  10%|█         | 2/20 [00:44<05:48, 19.35s/it]

...


PROCESSING TASKS | :  80%|████████  | 16/20 [04:03<00:48, 12.05s/it]

Superior Energy Services


PROCESSING TASKS | :  85%|████████▌ | 17/20 [04:29<00:49, 16.40s/it]

Washington Prime Group

PROCESSING TASKS | : 100%|██████████| 20/20 [05:10<00:00, 15.55s/it]
COLLECTING RESULTS | : 100%|██████████| 20/20 [00:00<00:00, 57026.57it/s]


In [133]:
import pandas as pd

In [136]:
results_nonerror = [r for r in results if type(r) == tuple]

In [137]:
#save the results
only_results = pd.DataFrame([r[1] for r in results_nonerror])
only_results.to_csv('../data/bankruptcy_dockets/output/only_results.csv')

In [138]:
#response details
details = pd.DataFrame([r[0] for r in results_nonerror])
details.to_csv('../data/bankruptcy_dockets/output/response_details.csv')

In [147]:
yes = []
for question_name,val in questions.items():
    print('question_name')
    print(only_results[f'{question_name}_correct'].value_counts())
    
    yes.append([question_name,only_results[only_results[f'{question_name}_correct'] == chatty_correct_type.Yes].shape[0]]) 


question_name
NameCorp_correct
chatty_correct_type.Yes    17
chatty_correct_type.No      2
Name: count, dtype: int64
question_name
AssetsPetition_correct
chatty_correct_type.Yes    15
chatty_correct_type.No      4
Name: count, dtype: int64
question_name
CeoFiling_correct
chatty_correct_type.No     11
chatty_correct_type.Yes     8
Name: count, dtype: int64
question_name
CityFiled_correct
chatty_correct_type.Yes    19
Name: count, dtype: int64
question_name
ClaimsUnsec_correct
chatty_correct_type.No    19
Name: count, dtype: int64
question_name
CommCred_correct
chatty_correct_type.Yes    14
chatty_correct_type.No      5
Name: count, dtype: int64
question_name
CommCredAtty_correct
chatty_correct_type.No     16
chatty_correct_type.Yes     3
Name: count, dtype: int64
question_name
CommEquity_correct
chatty_correct_type.Yes    16
chatty_correct_type.No      3
Name: count, dtype: int64
question_name
CommEquityAtty_correct
chatty_correct_type.No    19
Name: count, dtype: int64
question_name
Da

In [148]:
pd.DataFrame.from_dict(yes)

,0,1
0,NameCorp,17
1,AssetsPetition,15
2,CeoFiling,8
3,CityFiled,19
4,ClaimsUnsec,0
5,CommCred,14
6,CommCredAtty,3
7,CommEquity,16
8,CommEquityAtty,0
9,DateFiled,17


In [ ]:
results[1][1]

{'NameCorp': 'Basic Energy Services, Inc. (2021)',
 'NameCorp_response': {'val': 'Basic Energy Services Inc (2021)',
  'confidence': <confidence_type.High: 'high'>},
 'NameCorp_correct': 'Yes',
 'NameCorp_wny': "The human answer 'Basic Energy Services Inc (2021)' and the standard answer 'Basic Energy Services, Inc. (2021)' agree in principle. The differences are minor and do not change the meaning or identification of the entity.",
 'AssetsPetition': 331.0,
 'AssetsPetition_response': {'val': 331.0,
  'confidence': <confidence_type.High: 'high'>},
 'AssetsPetition_correct': 'Yes',
 'AssetsPetition_wny': 'The human answer and the standard answer both provide the value 331.0, indicating they agree in principle.',
 'CeoFiling': 'Keith L. Schilling',
 'CeoFiling_response': {'val': 'Adam L. Hurley, Executive Vice President, Chief Financial Officer, Treasurer and Secretary',
  'confidence': <confidence_type.High: 'high'>},
 'CeoFiling_correct': 'No',
 'CeoFiling_wny': "The standard answer is

In [ ]:
results[1][0]

{'NameCorp': 'Basic Energy Services, Inc. (2021)',
 'AssetsPetition': 331.0,
 'CeoFiling': 'Keith L. Schilling',
 'CityFiled': 'Houston',
 'ClaimsUnsec': nan,
 'CommCred': 'yes',
 'CommCredAtty': 'Brown Rudnick',
 'CommEquity': 'no',
 'CommEquityAtty': nan,
 'DateFiled': '8/17/2021',
 'DipAtty': 'Weil Gotshal',
 'Disposition': 'confirmed',
 'DistFiled': 'TX SD',
 'DistribEquity': nan,
 'DistribUnsec': nan,
 'Examiner': nan,
 'JudgeDisposition': 'David R. Jones',
 'JudgeFiling': 'David R. Jones',
 'LiabPetition': 549.0,
 'Prepackaged': 'free fall',
 'Sale': 'yes',
 'Sale363': 'yes',
 'SaleConf': nan,
 'TortCause': 'Not tort',
 'Trustee': nan,
 'TurnFirm': nan,
 'Voluntary': 'voluntary',
 'NameCorp_response': {'val': 'Basic Energy Services Inc (2021)',
  'confidence': <confidence_type.High: 'high'>},
 'NameCorp_val': 'Basic Energy Services Inc (2021)',
 'NameCorp_conf': 'high',
 'NameCorp_source_0': {'window': 'Box   \nFort Worth Texas 76102     \nCity  State  ZIP Code   City  State  ZIP